In [2]:
import bs4
import os
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import requests
from langchain_text_splitters import CharacterTextSplitter
from urllib.parse import urlparse, urljoin
from langchain_pinecone import Pinecone
from langchain_ai21 import AI21ContextualAnswers
from langchain_core.output_parsers import StrOutputParser
from langchain_pinecone import PineconeVectorStore

In [3]:
load_dotenv()

True

In [4]:
# Access the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
ai_twentyone_api_key = os.getenv('AI_TWENTYONE_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index_wit = os.getenv('PINECONE_WIT_STANDARD')

In [5]:
os.environ["AI21_API_KEY"] = ai_twentyone_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [6]:
stats_urls = [
	'https://www.niche.com/colleges/wentworth-institute-of-technology/',
	'https://www.appily.com/colleges/wentworth-institute-of-technology',
	'https://www.facebook.com/WentworthInst/',
	'https://www.bestcolleges.com/schools/wentworth-institute-of-technology/',
	'https://datausa.io/profile/university/wentworth-institute-of-technology',
	'https://www.collegevine.com/schools/wentworth-institute-of-technology-wit'

]

In [7]:
content = []

# Use Webloader to load and process each sub-url
for url in stats_urls:
    try:
        loader = WebBaseLoader(url)
        url_processed = loader.load()
        content.extend(url_processed)
        # Use loader.page_content here
    except Exception as e:
        continue

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=150)
splits = text_splitter.split_documents(content)
embeddings = OpenAIEmbeddings()

[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 273, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 440, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 301, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 408, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 277, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 293, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a chunk of size 22040, which is longer than the specified 250
[2024-06-02 15:28:27 - langchain_text_splitters.base - WARNING] Created a 

In [9]:
splits

[Document(page_content='Access to this page has been denied', metadata={'source': 'https://www.niche.com/colleges/wentworth-institute-of-technology/', 'title': 'Access to this page has been denied', 'description': 'px-captcha', 'language': 'en'}),
 Document(page_content="Wentworth Institute of Technology - Boston, MA | Appily   Skip to main content   Main navigation College Search Toggle the dropdown menu Search all Colleges Colleges by Ranking Toggle the dropdown menu Best Colleges by State Best Colleges by Major Colleges by Location Toggle the dropdown menu Search by City Search by State Colleges by Academics Toggle the dropdown menu ACT GPA SAT Colleges by Campus Resources Toggle the dropdown menu Cultural Student Resource Groups Disability Housing Available LGBTQIA Student Resource Groups Women’s Centers on Campus Colleges by Identity Group Toggle the dropdown menu Hispanic Serving HBCUs Women's Colleges Colleges by Admission Criteria Toggle the dropdown menu Test Optional Colleges

In [10]:
docsearch = PineconeVectorStore.from_documents(splits, embeddings, index_name=pc_index_wit)